In [1]:
import os 
import pandas as pd
import requests
import matplotlib.pyplot as plt
import requests
from pprint import pprint
import gmaps
from statsmodels.formula.api import ols
from config import Api_Key

ModuleNotFoundError: No module named 'config'

In [ ]:
# Configure gmaps
gmaps.configure(api_key=Api_Key)

In [ ]:
file = "Resources/Road_Crashes_for_five_Years_-_Victoria.csv"

main_df = pd.read_csv(file)

main_df.head()


In [ ]:
#Print a list of columns, so we can find out which column we need
print(main_df.columns.tolist())

In [ ]:
#Created a new DataFrame, and took out some of the columns. 
#Feel free to delete more, I didn't limit the columns to our discussed topics, as there maybe more ideas later and we may use the extra columns

crash_df = main_df[['accident_no',  'accident_date', 'date_value', 
                      'Daily_Rainfall', 'accident_time', 'alcoholtime', 
                      'accident_type', 'day_of_week', 'dca_code', 
                      'hit_run_flag', 'light_condition', 'police_attend', 
                      'road_geometry', 'severity', 'speed_zone', 
                      'run_offroad', 'longitude', 'latitude', 
                      'lga_name', 'region_name', 'total_persons', 
                      'inj_or_fatal', 'fatality', 'seriousinjury', 
                      'otherinjury', 'noninjured', 
                      'males', 'females',  
                      'alcohol_related','stat_div_name']]

In [ ]:
#Display the dataframe
crash_df.head()

In [ ]:
#---------Proposed topics---------
#-       Suburb vs accident numbers, display on Gmap (AJ)
#-       Days of the week vs Accident numbers (SK)
#-       Number of accident vs suburb (AJ)
#-       Number of accident vs Region 
#-       Light condition (groupby) vs number of accident... 
#                 -what else could be closely related to light condition (Corrolation/Linea regression)
#-       Nearest hospital vs suburbs that have the highest accident (Google Place API, Gmap) (JD)
#-       Multi-line plot for Type of accident, over the 5 years (AJ)
#-       Alcohol vs Dead end ["road_geometry"] (for fun!) (KK)
#-       Rainfall vs accident numbers (SK)
#-       Rainfall vs hit and run (KK)

In [ ]:
#-----Jing's Part-------

In [ ]:
#-       Nearest medical centre vs suburbs that have the highest accident (Google Place API, Gmap) (JD)

In [ ]:
#Create a new Dataframe for Jing's part of analayst, so it's not going to affect other team members work
suburb_df = crash_df[['accident_no',  'accident_date', 'date_value', 
                      'accident_type',  'dca_code', 
                      'light_condition','longitude', 'latitude', 
                      'lga_name', 'region_name']]

In [ ]:
#Remove rows without Longitude and Latitude (including those with "-1" value)
suburb_df_LatLng_cleaned = suburb_df.dropna()
suburb_df_LatLng_cleaned = suburb_df_LatLng_cleaned.loc[(suburb_df_LatLng_cleaned["longitude"] != -1)]

In [ ]:
suburb_df_LatLng_cleaned.head()

In [ ]:
#Create new dataframe for City vs Number of Accidents
suburb_accid_df = pd.DataFrame(suburb_df_LatLng_cleaned["lga_name"].value_counts())
suburb_accid_df=suburb_accid_df.rename(columns = {"lga_name" : "Number of Accident"})

In [ ]:
suburb_accid_df.head(50)

In [ ]:
#Add Latitude and Longitude of the cities
suburb_accid_df["Latitude"] = ""
suburb_accid_df["Longitude"] = ""
suburb_accid_df["Closest Medical Centre"] = ""
suburb_accid_df["Address"]=""

In [ ]:
#Display the new dataframe
suburb_accid_df.head(100)

In [ ]:
#Reset Index and create a list of City Names, with Victoria and AU
suburb_accid_df= suburb_accid_df.reset_index().copy()
city = suburb_accid_df["index"]+",Victoria,AU"

In [ ]:
#Loop throught the city names, and call GeoCode to return Latitude and Longtitude, then put into the datafram
index1=0
for x in city:
    
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?''address={0}&key={1}').format(x, Api_Key)
    
    geo_data = requests.get(target_url).json()
    
    results = geo_data["results"]
    
    suburb_accid_df.loc[index1, "Latitude"] = results[0]["geometry"]["location"]["lat"]
    suburb_accid_df.loc[index1, "Longitude"] = results[0]["geometry"]["location"]["lng"]
    index1+=1

In [ ]:
#Display the dataframe
suburb_accid_df.head(10)

In [ ]:
locations = suburb_accid_df[["Latitude", "Longitude"]]
weights = suburb_accid_df["Number of Accident"]/500

In [ ]:
# Plot Heatmap to show the hotspots in Victoria
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=2,
                                 point_radius=0.015)

#Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
#Create a new Dataframe to find out the top 10 cities with most number of accidents
first_10_df = suburb_accid_df.head(10).copy()
first_10_city_df = first_10_df[["index", "Number of Accident"]]

In [ ]:
#Plot the information into a bar chat
first_10_chart =first_10_city_df.plot(kind="bar", 
                                 grid=True, 
                                 title="Victorian City vs Number of Accidents (2014 to 2019)", 
                                 figsize=(10,7))
first_10_chart.set_xlabel("City Name")
first_10_chart.set_ylabel("Number of Accidents")
first_10_chart.set_xticklabels(first_10_city_df["index"], rotation = 45)
plt.tight_layout()

In [ ]:
# geocoordinates
target_radius = 5000
target_type = "hospital"

# set up a parameters dictionary
params = {"radius": target_radius,
    "type": target_type,
    "key": Api_Key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params["location"]= "-37.9848, 145.214"

# run a request using our params dictionary, test the API works
response = requests.get(base_url, params=params)
response_json=response.json()
pprint(response_json)


In [ ]:
# Loop throught the dataframe, and call the API to find the nearst medical center in the city
for index, row in first_10_df.iterrows():
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params)
    response_json=response.json()
    results = response_json["results"]
    #pprint(results)
    #print(index)
    
    
    try:
        first_10_df.loc[index, "Closest Medical Centre"] = results[0]["name"]
        first_10_df.loc[index, "Address"] = results[0]["vicinity"]
        print("Finding nearst medical center in " + row["index"])
    except:
        print(row["index"] + " not found!")
        
    print("============================")

In [ ]:
#Display the dataframe with the medical centre name
first_10_df.head(10)

In [ ]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Closest Medical Centre}</dd>
<dt>Address</dt><dd>{Address}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
medical_info1 = [info_box_template.format(**row) for index, row in first_10_df.iterrows()]
locations = first_10_df[["Latitude", "Longitude"]]

In [ ]:
city = first_10_df["Number of Accident"] / 100

In [ ]:
# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
#heat_layer = gmaps.heatmap_layer(locations, weights=city, 
#                                 dissipating=False, max_intensity=1000,
#                                 point_radius=0.01)

#Add layer
#fig.add_layer(heat_layer)

# Create symbol layer

symbol_layer = gmaps.marker_layer(locations, hover_text='', 
                      info_box_content=medical_info1 , display_info_box=None, label='')



fig.add_layer(symbol_layer)

# Display figure
fig

In [ ]:
#========End of Jing's part============

In [ ]:
#======AJ's part =============

In [ ]:
#Creating pie for all the 5 years for the accident types
accident_group = crash_df.groupby('accident_type')['accident_type'].count()
accident_chart = accident_group.plot(kind="pie", y='accident_type', title=("Accident Type Over 5 Years"), 
                                     legend=False, startangle=110, shadow=False, fontsize=10, figsize=(15,9))
plt.savefig("Accident Type Over 5 Years.png")

In [ ]:
#Select only collumn required for Jing's part of analaysis
light_df = crash_df[['accident_no',  'accident_date', 'date_value', 
                      'accident_type',  'dca_code', 
                      'light_condition','longitude', 'latitude', 
                      'lga_name', 'region_name']]
light_df

In [ ]:
light_accid_df = pd.DataFrame(light_df["light_condition"].value_counts())
light_accid_df

In [ ]:
light_accid_df=light_accid_df.rename(columns = {"light_condition" : "No of Accident"})

In [ ]:
light_accid_df.reset_index()

In [ ]:
light_chart =light_accid_df.plot(kind="bar", 
                                 grid=True, 
                                 title="Light Condition vs Number of Accidents (2014 to 2019)", 
                                 figsize=(10,7))
light_chart.set_xlabel("Light Condition")
light_chart.set_ylabel("Number of Accidents")
#plt.tight_layout()

In [ ]:
#creating a dataframe for the region
region_df = crash_df[['accident_no',  'accident_date', 'date_value', 
                      'accident_type',  'dca_code', 
                      'light_condition','longitude', 'latitude', 
                      'lga_name', 'region_name']]

In [ ]:
#creating new dataframe for the region based on the value_counts
region_accid_df = pd.DataFrame(region_df["region_name"].value_counts())
region_accid_df

In [ ]:
#renaming the region name column
region_accid_df= region_accid_df.rename(columns = {"region_name" : "No of Accidents"})

In [ ]:
#plotting on a bar graph 
region_chart =region_accid_df.plot(kind="bar", grid=True, 
                                 title="region vs Number of Accidents (2014 to 2019)", 
                                 figsize=(10,7))
region_chart.set_xlabel("Region")
region_chart.set_ylabel("Number of Accidents")
plt.savefig("region vs Number of Accidents (2014 to 2019.png")

In [ ]:
#creating a new dataframe to show accident_type by region
new_df = crash_df[['accident_no',  'accident_date', 'date_value', 
                      'accident_type',  'dca_code', 
                      'light_condition','longitude', 'latitude', 
                      'lga_name', 'region_name']]


In [ ]:
#grouping the dataframe by accident_type and getting the values for each region
new_region_accid_df = crash_df.groupby("accident_type")['region_name'].value_counts()
new_region_accid_df.head(70)

In [ ]:
#creating a dataframe for suburbs
suburb_df = crash_df[['accident_no',  'accident_date', 'date_value', 
                      'accident_type',  'dca_code', 
                      'light_condition','longitude', 'latitude', 
                      'lga_name', 'region_name']]

In [ ]:
#decalring accident_date as datetime.
suburb_df['accident_date'] =  pd.to_datetime(suburb_df['accident_date'], infer_datetime_format=True)


In [ ]:
#extracting the year from the accident_date column and creating a year column.
suburb_df["year"] = suburb_df['accident_date'].map(lambda x: x.year)
total_accidents = suburb_df.groupby('year')["accident_type"].count()
suburb_df

In [ ]:
accident_chart =total_accidents.plot(kind="bar", 
                                 grid=True, 
                                 title="Total Accidents over 5 years", 
                                 figsize=(15,5))
accident_chart.set_xlabel("year")
accident_chart.set_ylabel("Number of Accidents")

In [ ]:
new_suburb = suburb_df.loc[suburb_df["year"] != 2014]
new_suburb_02 = new_suburb.loc[new_suburb["year"] != 2019]

In [ ]:
#creating accident_rate by groupping the df by year. And getting keys for the years.
accident_rate = new_suburb_02.groupby('year')["accident_type"].count()/ 100
years = accident_rate .keys()
years
accident_rate

In [ ]:
#accident_rate over the 5 year period 
plt.plot(years, accident_rate)
plt.title("Accidents rate over the 5years")
plt.xlabel("years")
plt.ylabel("rate")
plt.savefig("Accidents rate over the 5years.png")

In [ ]:
#created a regression between years and the accident rate but not sure wheather this is correct. 
vc_slope, vc_int, vc_r, vc_p, vc_std_err = stats.linregress(years, accident_rate)
vc_fit =  years * vc_slope + vc_int
plt.scatter(years, accident_rate)
plt.plot(years, vc_fit)
line_eq = "y =" + str(round(vc_slope,2)) + "x +" + str(round(vc_int,2))
plt.annotate(line_eq,(2016, 140), fontsize=15, color="red")
print(f"The r-squared is: {vc_r**2}")
plt.title("Accidents rate over the 5years")
plt.xlabel("years")
plt.ylabel("rate")
plt.savefig("Accidents rate over the 5years_regression.png")
plt.show()

In [ ]:
model = ols('Number of Accident ~ Latitude +  Longitude', data=first_10_df)
fitted_model = model.fit()
fitted_model.summary()

In [ ]:
#Start of Stuart's Part********************************************

In [ ]:
import os 
import pandas as pd
import gmaps
import requests
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
%matplotlib inline

In [ ]:
# Weekday vs accident df
weekday_df = crash_df["day_of_week"].value_counts()
weekday_df

In [ ]:
# Weekday vs accident "Left Handed"

weekday_chart =weekday_df.plot(kind="bar",
                                 grid=True,
                                 title="2014 to 2019 - Did you have a Tough Week?",
                                 figsize=(10,7), color='r')

plt.xlabel("Day of the Week")
plt.ylabel("Number of Accidents")
plt.show()

In [ ]:
# Weekday vs accident "Right Handed"

plt.style.use('ggplot')
days = ["Saturday", "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
g = [8509, 10170, 10558, 11033, 11385, 11676, 11822]
x_pos = np.arange(len(days))
plt.bar(x_pos, g, color='r')
plt.xlabel("Days")
plt.ylabel("Number of Accidents")
plt.title("2014 to 2019 - Did You Have a Tough Week?")
plt.xticks(x_pos, days, rotation=45)
plt.tight_layout()
plt.savefig("../Bootcamp_ProjectOne/A Tough Week Jul_2014 to Jul_2019.png")

In [ ]:
#***************YEAR BY YEAR*************************************************************************

In [ ]:
#2015 Data Frame
fifteen_df_start = main_df.loc[main_df["date_value"] >=42005]
fifteen_df = fifteen_df_start.loc[fifteen_df_start["date_value"] <=42369]
fifteen_df.head()

In [ ]:
#2015 "day_of_week" Data Frame 
fifteendays_df = fifteen_df["day_of_week"].value_counts()
fifteendays_df

In [ ]:
#2016 Data Frame
sixteen_df_start = main_df.loc[main_df["date_value"] >=42370]
sixteen_df = sixteen_df_start.loc[sixteen_df_start["date_value"] <=42735]
sixteen_df.head()

In [ ]:
#2016 "day_of_week" Data Frame 
sixteendays_df = sixteen_df["day_of_week"].value_counts()
sixteendays_df

In [ ]:
#2017 Data Frame
sevteen_df_start = main_df.loc[main_df["date_value"] >=42736]
sevteen_df = sevteen_df_start.loc[sevteen_df_start["date_value"] <=43100]
sevteen_df.head()

In [ ]:
#2017 "day_of_week" Data Frame 
sevteendays_df = sevteen_df["day_of_week"].value_counts()
sevteendays_df

In [ ]:
#2018 Data Frame
eigteen_df_start = main_df.loc[main_df["date_value"] >=43101]
eigteen_df = eigteen_df_start.loc[eigteen_df_start["date_value"] <=43465]
eigteen_df.head()

In [ ]:
#2018 "day_of_week" Data Frame 
eigteendays_df = eigteen_df["day_of_week"].value_counts()
eigteendays_df

In [ ]:
daysx = (["Friday", "Thursday", "Wednesday", "Tuesday", "Monday", "Sunday", "Saturday"])
Y2015 = [2603, 2677, 2426, 2360, 2239, 2307, 1962]
Y2016 = [2634, 2481, 2560, 2392, 2335, 2293, 1816]
Y2017 = [2206, 2147, 2089, 2128, 2039, 1871, 1509]
Y2018 = [2057, 2070, 2064, 1972, 1926, 1703, 1491]

In [ ]:
# Generate a facet plot of all 4 figures
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True)
fig.suptitle('Accidents Every Day of the Week', fontsize=16, fontweight="bold")


ax1.set_xlim(min(daysx), max(daysx))
ax1.plot(daysx, Y2015, linewidth=1, marker="o")
#ax1.plot(daysx, Y2015, "b--", linewidth=1)
ax1.set_ylabel("2015")

ax2.plot(daysx, Y2016, linewidth=1, marker="o", color="r")
#ax2.plot(daysx, Y2016, "r--", linewidth=1)
ax2.set_ylabel("2016")

ax3.plot(daysx, Y2017, linewidth=1, marker="o", color="g")
#ax3.plot(daysx, Y2017, "g--", linewidth=1)
ax3.set_ylabel("2017")

ax4.plot(daysx, Y2018, linewidth=1, marker="o", color="b")
#ax4.plot(daysx, Y2018, "b--", linewidth=1)
ax4.set_ylabel("2018")
ax4.set_xlabel("Day of the Week")

plt.show()

In [ ]:
# Array rainfall amounts and count of accidents - "just playing with numbers"
rainfall_df = pd.DataFrame({
    "Rainfall":[0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1, 1.1, 1.2, 1.4, 1.6, 1.7, 1.8, 2, 2.2, 2.3, 2.4, 2.5, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.7, 3.8, 4, 4.2, 4.4, 4.5, 4.6, 4.8, 4.9, 5, 5.2, 5.4, 5.5, 5.6, 5.8, 6, 6.2, 6.4, 6.6, 6.8, 7, 7.2, 7.4, 7.6, 7.8, 8, 8.2, 8.4, 8.5, 8.6, 8.8, 9.2, 9.6, 9.7, 9.8, 10, 10.2, 10.4, 10.6, 10.8, 11, 11.2, 11.4, 11.6, 11.8, 12, 12.2, 12.6, 12.8, 13, 13.6, 14.2, 14.4, 15, 15.2, 15.6, 15.8, 16, 16.2, 16.4, 16.6, 16.8, 17.4, 17.6, 17.8, 18, 18.6, 18.8, 19.2, 19.6, 20.6, 20.8, 21, 21.2, 21.6, 21.9, 22, 22.6, 22.8, 23.2, 25.5, 25.6, 27, 27.8, 28, 28.6, 29.6, 30.6, 35, 37.2, 38.4, 41.8, 47],
    "Accidents":[120,2943,50,1697,1474,1331,1161,35,1303,644,767,26,947,551,903,34,511,41,809,680,475,164,602,436,22,24,290,303,290,51,266,120,34,214,165,236,34,330,90,41,234,170,29,181,330,216,137,108,70,105,139,104,42,41,149,76,60,38,212,41,67,44,121,182,81,90,49,41,44,44,125,83,105,92,51,50,49,126,30,38,33,41,71,45,93,51,32,47,36,34,91,31,45,44,45,51,54,39,52,29,47,38,39,50,42,86,48,37,52,48,45,44,20,44,44,36,43]
})
rainfall_df.head()

In [ ]:
# Array numbers into a scatter plot
x_limit = 47
x_axis = [0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1, 1.1, 1.2, 1.4, 1.6, 1.7, 1.8, 2, 2.2, 2.3, 2.4, 2.5, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.7, 3.8, 4, 4.2, 4.4, 4.5, 4.6, 4.8, 4.9, 5, 5.2, 5.4, 5.5, 5.6, 5.8, 6, 6.2, 6.4, 6.6, 6.8, 7, 7.2, 7.4, 7.6, 7.8, 8, 8.2, 8.4, 8.5, 8.6, 8.8, 9.2, 9.6, 9.7, 9.8, 10, 10.2, 10.4, 10.6, 10.8, 11, 11.2, 11.4, 11.6, 11.8, 12, 12.2, 12.6, 12.8, 13, 13.6, 14.2, 14.4, 15, 15.2, 15.6, 15.8, 16, 16.2, 16.4, 16.6, 16.8, 17.4, 17.6, 17.8, 18, 18.6, 18.8, 19.2, 19.6, 20.6, 20.8, 21, 21.2, 21.6, 21.9, 22, 22.6, 22.8, 23.2, 25.5, 25.6, 27, 27.8, 28, 28.6, 29.6, 30.6, 35, 37.2, 38.4, 41.8, 47],
data = [120,2943,50,1697,1474,1331,1161,35,1303,644,767,26,947,551,903,34,511,41,809,680,475,164,602,436,22,24,290,303,290,51,266,120,34,214,165,236,34,330,90,41,234,170,29,181,330,216,137,108,70,105,139,104,42,41,149,76,60,38,212,41,67,44,121,182,81,90,49,41,44,44,125,83,105,92,51,50,49,126,30,38,33,41,71,45,93,51,32,47,36,34,91,31,45,44,45,51,54,39,52,29,47,38,39,50,42,86,48,37,52,48,45,44,20,44,44,36,43]
plt.title("Rainfall Amount Vs Number of Accidents Jul_2014 to Jul_2019")
plt.xlabel("Rainfall in mm")
plt.ylabel("Number of Accidents")
plt.grid()
plt.scatter(x_axis, data, marker="o", facecolors="red", edgecolors="black",
            s=x_axis, alpha=0.75)

In [ ]:
rain_crash_df = main_df[['accident_no',  'accident_date', 'date_value', 
                      'Daily_Rainfall']]
rain_crash_df 

In [ ]:
accident_count = rain_crash_df["accident_no"].value_counts()
accident_count

In [ ]:
# Additional data set of rainfall data from BOM included in DoT data
file = "Resources/rain_vs_accident_five_years.csv"

rain_vs_acc = pd.read_csv(file)

rain_vs_acc.head()

In [ ]:
# Lets have a look
plt.scatter(rain_vs_acc.iloc[:,3],rain_vs_acc.iloc[:,2])
plt.ylabel('Daily Rainfall Amount (mm)')
plt.xlabel('Count of accidents')
plt.show()

In [ ]:
#Import dependency
import scipy.stats as st

#Pearson correlation coefficient between "Daily Rainfall Amount" and "Number of Accidents"
dailyrainfall = rain_vs_acc.iloc[:,3]
countofcrash = rain_vs_acc.iloc[:,2]
correlation = st.pearsonr(dailyrainfall,countofcrash)
print(f"The correlation between both factors is {round(correlation[0],2)}")

In [ ]:
#Lets turn it on it's side O-O

dailyrainfall = rain_vs_acc.iloc[:,2]
countofcrash = rain_vs_acc.iloc[:,3]
plt.scatter(dailyrainfall,countofcrash)
plt.xlabel('Daily Rainfall Amount (mm)')
plt.ylabel('Count of accidents')

#Pearson correlation coefficient between "Daily Rainfall Amount" and "Number of Accidents"
correlation = st.pearsonr(dailyrainfall,countofcrash)
print(f"The correlation between both factors is {round(correlation[0],2)}")
plt.show()

In [ ]:
#create the correlation df
corr_rainacc = rain_vs_acc[['daily_Rainfall_amount', 'count_of_accidents']]
corr_rainacc.head()

In [ ]:
#Rainfall vs count of accident correlation matrix to see (lack of) correlations
rain_corr = corr_rainacc.corr()
rain_corr.unstack().sort_values()

In [ ]:
# 'For Fun' all data correlation matrix "nothing to see here"
all_corr = main_df.corr()
all_corr.unstack().sort_values()

In [ ]:
#The final scatter

#Dependency
from scipy.stats import linregress

dailyrainfall = rain_vs_acc.iloc[:,3]
countofcrash = rain_vs_acc.iloc[:,2]
plt.scatter(dailyrainfall,countofcrash, color="b")
plt.xlabel('Count of accidents')
plt.ylabel('Daily Rainfall Amount (mm)')
plt.title('*July2014-July2019 Daily Rainfal Qty vs Count of Accident*')

#linear regression equation and line to plot
(slope, intercept, rvalue, pvalue, stderr) = linregress(dailyrainfall, countofcrash)
regress_values = dailyrainfall * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(dailyrainfall,regress_values,"r-")
plt.annotate(line_eq,(50,40),fontsize=15,color="r")

#Pearson correlation coefficient between "Daily Rainfall Amount" and "Number of Accidents"
correlation = st.pearsonr(dailyrainfall,countofcrash)
print(f"The correlation between both factors is {round(correlation[0],2)}")

# Print out the r-squared value 
print(f"The r-squared is: {rvalue**2}")
plt.show()
plt.savefig("../Bootcamp_ProjectOne/Rainfall Vs Accidents Jul_2014 to Jul_2019.png")

In [ ]:
# End of Stuart's Code........................................Have a nice day 